In [21]:
# Import reusable path setup
from setup_paths import setup_project_paths

# Get project paths
paths = setup_project_paths()

RAW_DIR = paths["RAW_DIR"]
PROCESSED_DIR = paths["PROCESSED_DIR"]

In [14]:
import os
import pandas as pd
from googleapiclient.discovery import build
from dotenv import load_dotenv

In [15]:
# Load API key from .env
load_dotenv()
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")


In [16]:
# Ensure data folders exist
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

In [17]:
# Initialize YouTube API
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

In [18]:
# Function to fetch trending videos
def fetch_trending_videos(region='US', max_results=50):
    request = youtube.videos().list(
        part='snippet,statistics',
        chart='mostPopular',
        regionCode=region,
        maxResults=max_results
    )
    response = request.execute()
    videos = []
    for item in response['items']:
        videos.append({
            'video_id': item['id'],
            'title': item['snippet']['title'],
            'channel': item['snippet']['channelTitle'],
            'views': int(item['statistics'].get('viewCount', 0)),
            'likes': int(item['statistics'].get('likeCount', 0)),
            'comments': int(item['statistics'].get('commentCount', 0)),
            'published_date': item['snippet']['publishedAt'],
            'category_id': item['snippet'].get('categoryId', None)
        })
    df = pd.DataFrame(videos)
    df.to_csv('data/raw/trending_videos.csv', index=False)
    return df


In [19]:
# Check if CSV exists
csv_path = 'data/raw/trending_videos.csv'
if os.path.exists(csv_path):
    print(f"{csv_path} already exists. Loading existing file...")
    df_raw = pd.read_csv(csv_path)
else:
    print("CSV not found. Fetching data from YouTube API...")
    df_raw = fetch_trending_videos()

df_raw.head()

data/raw/trending_videos.csv already exists. Loading existing file...


,video_id,title,channel,views,likes,comments,published_date,category_id
0,3cT5ML2l7KQ,VonOff1700 - Seen First (Official Video),VonOff1700,134175,10464,411,2025-10-21T19:00:02Z,10
1,i36Zw32GfRQ,Reminders of Him | Official Trailer,Universal Pictures,5230773,10188,483,2025-10-21T15:00:23Z,24
2,9_ofCQ0eOTc,Taco Tuesday Admin Abuse + New BRAINROTS!,CaylusBlox,2008316,32729,992,2025-10-21T23:07:13Z,20
3,pCv0oP9JLKw,Morgan Wallen - 20 Cigarettes (Official Music ...,MorganWallenVEVO,601989,22640,1864,2025-10-20T18:01:00Z,10
4,2DuX1l-clGY,Honest Trailers | The Thing,Screen Junkies,174906,12822,1075,2025-10-21T17:00:03Z,1
